# Train SHARE Model

## Train SAUTE model with MLM Loss

DO NOT RUN IN LOCAL

In [2]:
!mkdir sources
!curl https://raw.githubusercontent.com/Just1truc/share-qa/refs/heads/main/sources/datasets.py -o sources/datasets.py
!curl https://raw.githubusercontent.com/Just1truc/share-qa/refs/heads/main/sources/saute_model.py -o sources/saute_model.py
!curl https://raw.githubusercontent.com/Just1truc/share-qa/refs/heads/main/sources/saute_config.py -o sources/saute_config.py
!curl https://raw.githubusercontent.com/Just1truc/share-qa/refs/heads/main/sources/mlm_logger.py -o sources/mlm_logger.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6881  100  6881    0     0  18597      0 --:--:-- --:--:-- --:--:-- 18597
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 72907  100 72907    0     0   172k      0 --:--:-- --:--:-- --:--:--  172k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1507  100  1507    0     0   3419      0 --:--:-- --:--:-- --:--:--  3417
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3922  100  3922    0     0   8000      0 --:--:-- --:--:-- --:--:--  8004


You might need to restart session to actualize jupiter notebook env here

### Installing dependencies

In [1]:
%pip install flash-attn==1.0.8 --no-build-isolation
%pip install -U transformers
%pip install datasets
%pip install torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.2 MB/s eta 0:0

#### Imports

In [3]:
import wandb
wandb.init(project="best-model-dont-even-try")

import torch

from transformers         import Trainer, TrainingArguments, BertConfig, BertForMaskedLM, BertTokenizerFast
from sources.saute_model  import UtteranceEmbedings
from sources.saute_config import SAUTEConfig
from sources.datasets     import SAUTEDataset
from sources.mlm_logger   import WandbPredictionLoggerCallback
from torchinfo            import summary

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: justinduc (justinduc-epitech) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


#### Accuracy Metric

In [4]:
def compute_masked_accuracy(logits, labels):
    # logits: [batch_size, seq_len, vocab_size]
    # labels: [batch_size, seq_len]

    preds = torch.argmax(logits, dim=-1)  # [batch_size, seq_len]

    # Only consider masked positions (labels != -100)
    mask = labels != -100

    # Count correct predictions
    correct = (preds == labels) & mask
    accuracy = correct.sum().float() / mask.sum()

    return accuracy.item()

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    logits = torch.tensor(logits)
    labels = torch.tensor(labels)
    acc = compute_masked_accuracy(logits, labels)
    return {"masked_accuracy": acc}

#### Load Dataset

In [5]:
train_dataset = SAUTEDataset(split="train", dialog_format="edu")
eval_dataset = SAUTEDataset(split="test", dialog_format="edu")

from torch.utils.data import Subset
import random

subset_size = 10
indices = random.sample(range(len(eval_dataset)), subset_size)
test_dataset = Subset(eval_dataset, indices)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

...

valid.parquet:   0%|          | 0.00/82.9M [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1191582 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/146346 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/148968 [00:00<?, ? examples/s]

#### Load Model

In [6]:
model_config = SAUTEConfig(
    num_attention_heads = 12,
    num_hidden_layers   = 6,
    num_token_layers=3,
)
model = UtteranceEmbedings(model_config).to("cuda:0")

#### Setup Data Collator

In [7]:
def batched_saute_collator(batch):

    pad_token = "[PAD]"

    input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]
    attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
    speaker_names_list = [x["speaker_names"] for x in batch]
    labels_list        = [torch.tensor(x["labels"]) for x in batch]

    max_T = max(len(x) for x in speaker_names_list)
    max_L = max(seq.shape[1] for seq in input_ids_list)

    def pad_dialog(tensor, max_T, max_L, pad_val):
        return torch.nn.functional.pad(tensor, (0, max_L - tensor.shape[1], 0, max_T - tensor.shape[0]), value=pad_val)

    input_ids = torch.stack([pad_dialog(seq, max_T, max_L, tokenizer.pad_token_id) for seq in input_ids_list])
    attention_mask = torch.stack([pad_dialog(mask, max_T, max_L, 0) for mask in attention_masks])
    labels = torch.stack([pad_dialog(lbl, max_T, max_L, -100) for lbl in labels_list])  # -100: ignore index for loss

    # print(len(input_ids_list))
    if len(input_ids_list) == 1:
      input_ids_list = [input_ids_list[0].unsqueeze(0)]
      attention_masks = [attention_masks[0].unsqueeze(0)]
      labels_list = [labels_list[0].unsqueeze(0)]


    speaker_names = [
        names + [pad_token] * (max_T - len(names))
        for names in speaker_names_list
    ]

    return {
        "input_ids": input_ids,               # (B, T, L)
        "attention_mask": attention_mask,     # (B, T, L)
        "speaker_names": speaker_names,       # (B, T)
        "labels": labels                      # (B, T, L)
    }

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


#### Training

In [8]:
fixed_batch = batched_saute_collator([train_dataset[0]])
print(fixed_batch["input_ids"].shape)

torch.Size([1, 6, 130])


<ipython-input-7-11281c7b3996>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]
<ipython-input-7-11281c7b3996>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
<ipython-input-7-11281c7b3996>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_list        = [torch.tensor(x["labels"]) for x in batch]


#### Model Statistics

In [7]:

class ModelWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, speaker_names=fixed_batch["speaker_names"])

wrapped_model = ModelWrapper(model).to("cuda:0")

summary(
    wrapped_model,
    input_data=(fixed_batch["input_ids"].to("cuda:0"), fixed_batch["attention_mask"].to("cuda:0")),
    col_names=["input_size", "output_size", "num_params", "mult_adds"],
    depth=3,
    verbose=1
)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
ModelWrapper                                                 [1, 6, 130]               [1, 6, 130, 30522]        --                        --
├─UtteranceEmbedings: 1-1                                    --                        [1, 6, 130, 30522]        --                        --
│    └─MemoryBankDiscourseTransformer: 2-1                   --                        --                        17,796,864                --
│    │    └─Embedding: 3-1                                   [1, 6, 130]               [1, 6, 130, 768]          23,440,896                23,440,896
│    │    └─Embedding: 3-2                                   [130]                     [130, 768]                393,216                   51,118,080
│    │    └─Embedding: 3-3                                   [1, 6]                    [1, 6, 768]               153,600     

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
ModelWrapper                                                 [1, 6, 130]               [1, 6, 130, 30522]        --                        --
├─UtteranceEmbedings: 1-1                                    --                        [1, 6, 130, 30522]        --                        --
│    └─MemoryBankDiscourseTransformer: 2-1                   --                        --                        17,796,864                --
│    │    └─Embedding: 3-1                                   [1, 6, 130]               [1, 6, 130, 768]          23,440,896                23,440,896
│    │    └─Embedding: 3-2                                   [130]                     [130, 768]                393,216                   51,118,080
│    │    └─Embedding: 3-3                                   [1, 6]                    [1, 6, 768]               153,600     

#### Init Training necessities

In [9]:
# fixed_batch = train_dataset[1]
wrapped_model = None
# Initialize the callback
wandb_logger_callback = WandbPredictionLoggerCallback(
    fixed_batch=fixed_batch,
    tokenizer=tokenizer,
    log_every_steps=50
)

In [ ]:
from transformers import DataCollatorWithPadding
from torch.nn.utils.rnn import pad_sequence
import torch

torch.autograd.set_detect_anomaly(True)

tokenizer_name = "bert-base-uncased"
training_args = TrainingArguments(
    output_dir="h-saute-mlm-76m-0.0.2",
    eval_strategy="steps",
    eval_steps=150,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=150,
    fp16=True
    # deepspeed="deepspeed_config.json",  # optional
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=None,
    # data_collator=lambda batch: batch,
    data_collator = batched_saute_collator,
    callbacks=[wandb_logger_callback],
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

<ipython-input-10-f26bbd7afb5c>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
<ipython-input-7-11281c7b3996>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]
<ipython-input-7-11281c7b3996>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks    = [torch.tensor(x["attention_mask"]) for x

Step,Training Loss,Validation Loss,Masked Accuracy
150,6.673600,5.174314,0.125000
300,5.410700,4.668328,0.207171
450,5.244100,4.918873,0.190265
600,4.994800,4.542239,0.296137
750,4.760300,4.636222,0.235043
900,4.583300,3.838406,0.348548
1050,4.434400,3.803123,0.355212
1200,4.324500,3.651541,0.364017
1350,4.133000,3.853040,0.340611
1500,3.961600,3.381334,0.369369


### Bert Baseline

#### Load Model and dataset

In [ ]:
train_dataset = SAUTEDataset(split="train", dialog_format="full")
eval_dataset = SAUTEDataset(split="test", dialog_format="full")

from torch.utils.data import Subset
import random

subset_size = 10
indices = random.sample(range(len(eval_dataset)), subset_size)
test_dataset = Subset(eval_dataset, indices)

bert_config = BertConfig(
    vocab_size=30522,
    hidden_size=768,
    num_hidden_layers=6,
    num_attention_heads=12,
    intermediate_size=3072,
    max_position_embeddings=512,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1
)
model = BertForMaskedLM(config=bert_config)

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

def batch_collator(batch):
    input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]         # List[(T_i, L_i)]
    attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
    labels_list        = [torch.tensor(x["labels"]) for x in batch]

    max_T = max(seq.shape[0] for seq in input_ids_list)
    max_L = max(seq.shape[1] for seq in input_ids_list)

    def pad_dialog(tensor, max_T, max_L, pad_val):
        return torch.nn.functional.pad(
            tensor, (0, max_L - tensor.shape[1], 0, max_T - tensor.shape[0]),
            value=pad_val
        )

    padded_input_ids = torch.stack([pad_dialog(seq, max_T, max_L, tokenizer.pad_token_id) for seq in input_ids_list])  # (B, T, L)
    padded_attention = torch.stack([pad_dialog(mask, max_T, max_L, 0) for mask in attention_masks])               # (B, T, L)
    padded_labels    = torch.stack([pad_dialog(lbl, max_T, max_L, -100) for lbl in labels_list])     # (B, T, L)

    # Flatten (B × T, L) to feed BERT
    B, T, L = padded_input_ids.shape
    flat_input_ids   = padded_input_ids.view(B * T, L)
    flat_attention   = padded_attention.view(B * T, L)
    flat_labels      = padded_labels.view(B * T, L)

    return {
        "input_ids": flat_input_ids,
        "attention_mask": flat_attention,
        "labels": flat_labels
    }

In [ ]:
fixed_batch = batch_collator([train_dataset[0]])

print(fixed_batch["input_ids"].shape)
print(fixed_batch["labels"].shape)
print(fixed_batch["attention_mask"].shape)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

wandb_logger_callback = WandbPredictionLoggerCallback(
    fixed_batch=fixed_batch,
    tokenizer=tokenizer,
    log_every_steps=10
)

<ipython-input-4-bdeb16a81fc4>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]         # List[(T_i, L_i)]
<ipython-input-4-bdeb16a81fc4>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
<ipython-input-4-bdeb16a81fc4>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_list        = [torch.tensor(x["labels"]) for x in batch]


torch.Size([6, 135])
torch.Size([6, 135])
torch.Size([6, 135])


#### Model Statistics

In [ ]:
summary(
    model,
    input_data=(fixed_batch["input_ids"], fixed_batch["attention_mask"]),
    col_names=["input_size", "output_size", "num_params", "mult_adds"],
    depth=3,
    verbose=1
)

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
BertForMaskedLM                                              [6, 135]                  [6, 135, 30522]           --                        --
├─BertModel: 1-1                                             [6, 135]                  [6, 135, 768]             --                        --
│    └─BertEmbeddings: 2-1                                   --                        [6, 135, 768]             --                        --
│    │    └─Embedding: 3-1                                   [6, 135]                  [6, 135, 768]             23,440,896                140,645,376
│    │    └─Embedding: 3-2                                   [6, 135]                  [6, 135, 768]             1,536                     9,216
│    │    └─Embedding: 3-3                                   [1, 135]                  [1, 135, 768]             393,216         

Layer (type:depth-idx)                                       Input Shape               Output Shape              Param #                   Mult-Adds
BertForMaskedLM                                              [6, 135]                  [6, 135, 30522]           --                        --
├─BertModel: 1-1                                             [6, 135]                  [6, 135, 768]             --                        --
│    └─BertEmbeddings: 2-1                                   --                        [6, 135, 768]             --                        --
│    │    └─Embedding: 3-1                                   [6, 135]                  [6, 135, 768]             23,440,896                140,645,376
│    │    └─Embedding: 3-2                                   [6, 135]                  [6, 135, 768]             1,536                     9,216
│    │    └─Embedding: 3-3                                   [1, 135]                  [1, 135, 768]             393,216         

#### Train model

In [ ]:
training_args = TrainingArguments(
    output_dir="batched-bert-baseline-mlm-edu-90m",
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    # save_strategy="steps",
    # save_steps=1000,
    eval_strategy="steps",
    eval_steps=150,
    logging_dir="./logs-bert",
    logging_steps=150,
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="wandb",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # data_collator=lambda batch: batch[0],
    data_collator=batch_collator,
    callbacks=[wandb_logger_callback],
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
<ipython-input-4-bdeb16a81fc4>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]         # List[(T_i, L_i)]
<ipython-input-4-bdeb16a81fc4>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
<ipython-input-4-bdeb16a81fc4>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.cl

Step,Training Loss,Validation Loss,Masked Accuracy
150,6.737900,5.351736,0.169421
300,5.355600,5.017028,0.163866
450,5.106800,4.936014,0.235772
600,4.920200,4.446258,0.269406
750,4.757200,4.571261,0.266667
900,4.512500,4.172335,0.300000
1050,4.357500,4.158114,0.350598
1200,4.195300,3.730671,0.365854
1350,4.066700,4.256622,0.341667
1500,3.845100,3.686763,0.372951


<ipython-input-4-bdeb16a81fc4>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_list     = [torch.tensor(x["input_ids"]) for x in batch]         # List[(T_i, L_i)]
<ipython-input-4-bdeb16a81fc4>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks    = [torch.tensor(x["attention_mask"]) for x in batch]
<ipython-input-4-bdeb16a81fc4>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_list        = [torch.tensor(x["labels"]) for x in batch]
<ipython-input-4-bdeb16a81fc4>:4: UserWarning: To copy construct from a te